## Procesamiento de los datos

### 0 - Configuracion Sagemaker

In [ ]:
import pandas as pd
import os

import boto3
import sagemaker

import tensorflow as tf
from tensorflow.keras.preprocessing import sequence

In [ ]:
# identificación de región
region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

# indetificación de rol
role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

# sesión de sagemaker
sagemaker_session = sagemaker.Session()

### 1 - Descarga y exploración de los datos

**Descarga de los datos**

https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data

In [ ]:
# número maximo del vocabulario, devuelve las n palabras más utilizadas
max_features = 20000
# máxima longitud de las reseñas
maxlen = 500

# descargamos los datos
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=max_features)

print(len(x_train), "train sequences")
print(len(x_test), "test sequences")

**Exploración de los datos**

In [ ]:
print(f" Número máximo de indice de las palabras: {max([max(review) for review in x_train])}")

In [ ]:
print(f"Frase de ejemplo: {x_train[0]}")

In [ ]:
# convertir palabras a indice
word_to_integer = tf.keras.datasets.imdb.get_word_index()
# ejemplo de palabras más comunes
print(list(word_to_integer.items())[0:5])

In [ ]:
# ejemplo de conversión
word_to_integer['the']

In [ ]:
# convertir indice a la palabra
integer_to_word = dict([(value, key) for (key, value) in word_to_integer.items()])
# ejemplo del índice a la palabra
integer_to_word[1]

In [ ]:
# Necesitamos substraer 3 al indice. El 0 es para el padding, el 1 para el inicio de la secuencia y el 2 para el unknown
# Cada review tiene una longitud diferente
decoded_review = ' '.join([integer_to_word.get(i - 3, 'DESCO') for i in x_train[0]])
print(decoded_review)

### 2 - Padding

In [ ]:
# misma longitud
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

### 3 - Save data

In [ ]:
# pasamos a dataframe
df_x_train = pd.DataFrame(x_train)
df_y_train = pd.DataFrame(y_train)
df_x_test = pd.DataFrame(x_test)
df_y_test = pd.DataFrame(y_test)

In [ ]:
# juntamos variables con resultados
df_train = pd.concat([df_y_train, df_x_train], axis=1)
df_test_and_validation = pd.concat([df_y_test, df_x_test], axis=1)

In [ ]:
# dívidimos el conjunto de test entre validation y test
df_validation = df_test_and_validation[:20000]
df_test = df_test_and_validation[20000:]

**En local**

In [ ]:
# Creamos el directorio
data_dir_train = "./data/pd/train"
if not os.path.exists(data_dir_train):
    os.makedirs(data_dir_train)

data_dir_test = "./data/pd/test"
if not os.path.exists(data_dir_test):
    os.makedirs(data_dir_test)

df_train.to_csv(os.path.join(data_dir_train, 'train.csv'), header=False, index=False)
df_validation.to_csv(os.path.join(data_dir_train, 'validation.csv'), header=False, index=False)
df_test.to_csv(os.path.join(data_dir_test, 'test.csv'), header=False, index=False)

**En S3**

In [ ]:
s3_folder_bucket = "clasificador_sentimiento/data"

train_s3 = sagemaker_session.upload_data(os.path.join(data_dir_train, 'train.csv'), key_prefix=f"{s3_folder_bucket}/train")
validation_s3 = sagemaker_session.upload_data(os.path.join(data_dir_train, 'validation.csv'), key_prefix=f"{s3_folder_bucket}/train")
test_s3 = sagemaker_session.upload_data(os.path.join(data_dir_test, 'test.csv'), key_prefix=f"{s3_folder_bucket}/test")


inputs = {"train": train_s3, "test": test_s3, "validation": validation_s3}
inputs